In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Change directory to your code
%cd /content/drive/MyDrive/addition
%pwd   # verify you’re in the right place
!ls    # should show train.py, 4_operands_addition.txt, etc.

Mounted at /content/drive
/content/drive/MyDrive/addition
configuration_files  mae_by_pred_iter.png  result_analysis.ipynb
configurator.py      main_utilities.py	   results
data		     model.py		   startHere.ipynb
evaluation.py	     __pycache__	   statistical_measurements.py
legacy_code	     README.md		   train.py


In [ ]:
!pip install torch tqdm pyyaml wandb pandas requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [ ]:
!pip install wandb

In [ ]:
# Paste your 40‑character WandB API key between the quotes:
!wandb login a4aba93ba2c1ed22804b69ecf18fa5a072b81c47

wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


## Reproduce 2 Operands 0-999 Addition (Their Code, Their Data)

### Plain

In [ ]:
%cat config2/addition/plain/train_addition_bal.py

# train plain addition on NanoGPT
dtype = 'float16'
init_from = 'scratch' # 'scratch' or 'resume' or 'gpt2*'
resume_iter = False # if True, resume from saved iter_num, otherwise resume from iter_num 0
ckpt_path_name = 'ckpt_teaching_addition_plain_10000.pt'
# ===== Evaluation and Checkpointing ===== #
out_dir = 'out2/addition_plain_more_early_eval'
eval_interval = 250 
eval_iters = 200
log_interval = 10 
always_save_checkpoint = False

# ===== Wandb logging ===== #
wandb_log = True # override via command line if you like
wandb_project = 'addition'
wandb_run_name = 'teaching_addition_plain_more_early_eval'

# ===== Dataset ===== #
data_type='text'
data_format='plain'
operator='+'
dataset = 'bal'
batch_size = 256
block_size = 256 # context of up to 256 previous characters
train_data_path = 'train_3digit_10000.txt'
eval_addition = True
start = "FILE:data/bal/test_10000.txt"
eval_addition_train = True

# ===== NanoGPT model configuration ===== #
n_layer = 6
n_head = 6
n_embd = 384
dropout 

In [ ]:
!python train.py config2/addition/plain/train_addition_bal.py

Streaming output truncated to the last 5000 lines.
outputs(x):  322+648=960
904
wrong  : 322+648=960
correct: 322+648=970
outputs(x):  571+826=1497
69
wrong  : 571+826=1497
correct: 571+826=1397
outputs(x):  429+372=701
362
wrong  : 429+372=701
correct: 429+372=801
 26% 21/80 [00:00<00:02, 25.74it/s]outputs(x):  817+383=1100
54
wrong  : 817+383=1100
correct: 817+383=1200
outputs(x):  382+418=700
0+9
wrong  : 382+418=700
correct: 382+418=800
outputs(x):  981+318=1399
27
wrong  : 981+318=1399
correct: 981+318=1299
outputs(x):  623+875=1598
21
wrong  : 623+875=1598
correct: 623+875=1498
outputs(x):  632+318=940
369
wrong  : 632+318=940
correct: 632+318=950
outputs(x):  533+163=796
389
wrong  : 533+163=796
correct: 533+163=696
outputs(x):  804+945=1759
87
wrong  : 804+945=1759
correct: 804+945=1749
outputs(x):  525+276=701
694
wrong  : 525+276=701
correct: 525+276=801
outputs(x):  195+112=207
297
wrong  : 195+112=207
correct: 195+112=307
outputs(x):  142+664=706
969
wrong  : 142+664=706
co

### Reverse

In [ ]:
%cat config2/addition/reverse/train_addition_bal.py

# train a miniature character-level shakespeare model
dtype = 'float16'
init_from = 'scratch' # 'scratch' or 'resume' or 'gpt2*'
resume_iter = False # if True, resume from saved iter_num, otherwise resume from iter_num 0
ckpt_path_name = 'ckpt_teaching_addition_reverse_10000.pt'
# good for debugging and playing on macbooks and such
out_dir = 'out2/addition_reverse_more_early_eval_v2'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = True # override via command line if you like
wandb_project = 'addition'
wandb_run_name = 'teaching_addition_reverse_more_early_eval_v2'

data_type='text'
data_format='reverse'
operator='+'
dataset = 'bal'
batch_size = 256
block_size = 256 # context of up to 256 previous characters
train_data_path = 'train_3digit_10000.txt'
# val_data_path = 'val.bin'
reverse_c = True


In [ ]:
!python train.py config2/addition/reverse/train_addition_bal.py

Streaming output truncated to the last 5000 lines.
wrong  : 704+386=1089
correct: 704+386=1090
outputs(x):  $902+606=7051$
$
wrong  : 902+606=1507
correct: 902+606=1508
outputs(x):  $826+877=3971$
$
wrong  : 826+877=1793
correct: 826+877=1703
outputs(x):  $350+759=9001$
$
wrong  : 350+759=1009
correct: 350+759=1109
outputs(x):  $101+506=606$
$9
wrong  : 101+506=606
correct: 101+506=607
outputs(x):  $282+729=119$
$4
wrong  : 282+729=911
correct: 282+729=1011
outputs(x):  $191+838=829$
$8
wrong  : 191+838=928
correct: 191+838=1029
outputs(x):  $323+600=328$
$7
wrong  : 323+600=823
correct: 323+600=923
outputs(x):  $262+729=198$
$1
wrong  : 262+729=891
correct: 262+729=991
outputs(x):  $353+756=8011$
$
wrong  : 353+756=1108
correct: 353+756=1109
outputs(x):  $134+963=7801$
$
wrong  : 134+963=1087
correct: 134+963=1097
outputs(x):  $217+805=229$
$7
wrong  : 217+805=922
correct: 217+805=1022
outputs(x):  $373+477=948$
$5
wrong  : 373+477=849
correct: 373+477=850
outputs(x):  $981+866=6481$


## Reproduce 2 Operands 0-999 Addition (Our Code, Their Data)

In [ ]:

# Change directory to your code
%cd /content/drive/MyDrive/addition
%pwd   # verify you’re in the right place
!ls    # should show train.py, 4_operands_addition.txt, etc.

/content/drive/MyDrive/addition
 0_to_99999_times_1_digit
 0_to_six_digit_times_1_digit
 2_operands_0_to_999_balanced_digit_plain
 2_operands_0_to_999_balanced_digit_reversed
 2_operands_0_to_999_output_wo_leading_digit
 2_operands_0_to_999_their_data
 2_operands_0_to_999_their_training_data_only_more_eval
 2_operands_0_to_999_their_training_our_testing
 2_operands_0_to_999_uniform
 2_operands_0_to_999_uniform_using_new_code
 2_operands_3_digit_output_1000+
 2_operands_3_digit_output_padding
 2_operands_addition_plain.txt
 2_operands_addition_reversed.txt
 2_operands_mul_plain.txt
 2_operands_mul_reversed.txt
 3_operands_addition.txt
 4_operands_0_to_999_balanced_digit
 4_operands_0_to_999_uniform_wo_padding
 4_operands_0_to_999_uniform_w_padding
 4_operands_3_digit_uniform_output_padding
 4_operands_addition_3_digit.txt
 4_operands_addition_plain.txt
 4_operands_addition_reversed.txt
 5_operands_addition.txt
 6_operands_0_to_999_balanced_digit
 6_operands_addition_reversed.txt
 7_oper

### Plain

In [ ]:
%cat ./data/2_operands_0_to_999_balanced_digit_original_data/test/test_10000_stripped.txt

0+10=
0+360=
0+536=
0+551=
0+646=
0+741=
1+116=
1+517=
1+685=
1+985=
2+16=
2+36=
2+115=
2+120=
2+197=
2+545=
2+650=
2+712=
2+817=
2+878=
2+899=
2+980=
3+15=
3+228=
3+229=
3+323=
3+442=
3+453=
3+457=
3+510=
3+553=
3+576=
3+635=
3+723=
3+849=
4+39=
4+233=
4+294=
4+353=
4+386=
4+447=
4+456=
4+511=
4+636=
4+644=
4+712=
4+787=
4+829=
4+919=
5+120=
5+172=
5+564=
5+902=
6+213=
6+301=
6+512=
6+555=
6+598=
6+631=
6+636=
6+791=
6+957=
6+989=
7+66=
7+106=
7+161=
7+250=
7+287=
7+499=
7+520=
7+699=
7+785=
7+869=
7+975=
7+977=
7+996=
7+999=
8+16=
8+118=
8+488=
8+572=
8+625=
8+758=
8+842=
9+21=
9+23=
9+31=
9+162=
9+201=
9+210=
9+219=
9+483=
9+507=
9+550=
9+617=
9+750=
9+845=
9+987=
10+42=
10+286=
10+622=
10+721=
10+967=
11+99=
11+149=
11+292=
11+355=
11+439=
11+598=
11+873=
11+982=
12+194=
12+375=
12+393=
12+476=
12+595=
12+638=
12+682=
12+738=
12+743=
12+982=
13+101=
13+213=
13+287=
13+354=
13+440=
13+579=
13+783=
13+929=
13+951=
13+993=
14+379=
14+448=
14+485=
14+495=
14+611=
14+624=
14+783=
15+58=

In [ ]:
!python train_end_padding_auto_val.py 2_operands_addition_plain.txt

Streaming output truncated to the last 5000 lines.
Skipping y_hat=219
Skipping y_hat=229
Skipping y_hat=260
Skipping y_hat=396
Skipping y_hat=305
Skipping y_hat=392
Skipping y_hat=209
Skipping y_hat=330
Skipping y_hat=211
Skipping y_hat=280
Skipping y_hat=300
Skipping y_hat=399
Skipping y_hat=393
Skipping y_hat=300
Skipping y_hat=348
Skipping y_hat=347
Skipping y_hat=397
Skipping y_hat=312
Skipping y_hat=204
Skipping y_hat=306
Skipping y_hat=300
Skipping y_hat=404
Skipping y_hat=340
Skipping y_hat=401
Skipping y_hat=491
Skipping y_hat=421
Skipping y_hat=493
Skipping y_hat=441
Skipping y_hat=497
Skipping y_hat=312
Skipping y_hat=448
Skipping y_hat=345
 14% 11/80 [00:00<00:02, 32.42it/s]Skipping y_hat=496
Skipping y_hat=509
Skipping y_hat=449
Skipping y_hat=582
Skipping y_hat=405
Skipping y_hat=405
Skipping y_hat=541
Skipping y_hat=515
Skipping y_hat=404
Skipping y_hat=405
Skipping y_hat=403
Skipping y_hat=514
Skipping y_hat=697
Skipping y_hat=695
Skipping y_hat=629
Skipping y_hat=694
Sk

### Reversed

In [ ]:
!python train_end_padding_auto_val.py 2_operands_addition_reversed.txt

Streaming output truncated to the last 5000 lines.
Skipping y_hat=139
Skipping y_hat=328
Skipping y_hat=449
Skipping y_hat=846
Skipping y_hat=769
Skipping y_hat=1044
Skipping y_hat=806
Skipping y_hat=321
Skipping y_hat=545
Skipping y_hat=265
Skipping y_hat=568
Skipping y_hat=773
Skipping y_hat=397
Skipping y_hat=101
Skipping y_hat=307
Skipping y_hat=412
Skipping y_hat=415
Skipping y_hat=337
Skipping y_hat=359
Skipping y_hat=487
Skipping y_hat=1000
Skipping y_hat=605
Skipping y_hat=415
Skipping y_hat=533
Skipping y_hat=443
Skipping y_hat=443
Skipping y_hat=450
Skipping y_hat=460
Skipping y_hat=624
Skipping y_hat=568
Skipping y_hat=306
Skipping y_hat=745
Skipping y_hat=461
Skipping y_hat=488
Skipping y_hat=451
Skipping y_hat=561
Skipping y_hat=581
Skipping y_hat=998
Skipping y_hat=815
Skipping y_hat=825
Skipping y_hat=541
Skipping y_hat=575
Skipping y_hat=677
Skipping y_hat=893
Skipping y_hat=1001
Skipping y_hat=728
Skipping y_hat=951
Skipping y_hat=963
Skipping y_hat=467
Skipping y_hat=

## Reproduce 2 Operands 0-999 Addition (Our Code, Their Training Data Only)



### Plain

In [ ]:
%cat 2_operands_addition_plain.txt

# to edit: do an evaluation every {eval_interval} iterations
eval_interval = 100
eval_iters = 1
always_save_checkpoint = True


wandb_log = True # override via command line if you like
wandb_project = 'addition'

# to edit: wandb run name
wandb_run_name = 'our_model_their_training_data_only_plain_more_early_eval'

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

# to edit: 'plain' or 'reverse' or 'algo_reasoning'
data_format='plain'
operator='+'
dataset = 'bal'
batch_size = 512
block_size = 32 # context of up to 256 previous characters

# to edit: 'scratch' or 'resume', whether train from scratch or resume from a saved checkpoint
init_from = 'scratch'
# to edit: the checkpoint from which to resume training
ckpt_path_name = 'ckpt_2_operands_0_to_999_their_training_data_plain.pt'

# to edit: whether the result is reversed
reverse_c = False
eval_addition = True

analysis = False

# to edit: the number of addition operations to perform, which equals to (#operands -1)


In [ ]:
!python train_end_padding_more_early_eval.py 2_operands_addition_plain.txt

Streaming output truncated to the last 5000 lines.
Skipping y_hat=899
Skipping y_hat=687
Skipping y_hat=457
Skipping y_hat=1008
Skipping y_hat=725
Skipping y_hat=626
Skipping y_hat=486
Skipping y_hat=43
Skipping y_hat=109
Skipping y_hat=861
Skipping y_hat=846
Skipping y_hat=839
Skipping y_hat=522
Skipping y_hat=627
Skipping y_hat=381
Skipping y_hat=486
Skipping y_hat=140
Skipping y_hat=633
Skipping y_hat=806
Skipping y_hat=230
Skipping y_hat=980
Skipping y_hat=752
Skipping y_hat=337
Skipping y_hat=863
Skipping y_hat=594
Skipping y_hat=413
Skipping y_hat=482
Skipping y_hat=437
Skipping y_hat=264
Skipping y_hat=571
Skipping y_hat=848
Skipping y_hat=426
Skipping y_hat=679
Skipping y_hat=539
Skipping y_hat=238
Skipping y_hat=206
Skipping y_hat=268
Skipping y_hat=682
Skipping y_hat=333
Skipping y_hat=938
Skipping y_hat=110
Skipping y_hat=485
Skipping y_hat=329
Skipping y_hat=251
Skipping y_hat=456
Skipping y_hat=880
Skipping y_hat=861
Skipping y_hat=915
Skipping y_hat=1002
Skipping y_hat=60

## 4 Operands 0-999 Addition (Our Generated Balanced Digit Data)

### Plain

In [ ]:
%cat 4_operands_addition_plain.txt

# to edit: do an evaluation every {eval_interval} iterations
eval_interval = 1000
eval_iters = 1
always_save_checkpoint = True


wandb_log = True # override via command line if you like
wandb_project = 'addition'

# to edit: wandb run name
wandb_run_name = '4_operands_0_to_999_balanced_digit_plain_v4'

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

# to edit: 'plain' or 'reverse' or 'algo_reasoning'
data_format='plain'
operator='+'
dataset = 'bal'
batch_size = 512
block_size = 32 # context of up to 256 previous characters

# to edit: the checkpoint from which to resume training
ckpt_path_name = 'ckpt_iter_19000_acc.pt'

# to edit: whether the result is reversed
reverse_c = False
eval_addition = True

analysis = False

# to edit: the number of addition operations to perform, which equals to (#operands -1)
num_addition = 3

# to edit: num of digits in each operand
num_digit = 3

# to edit: maximum number of output tokens (including '$')
max_new_tokens = 5

# wheth

In [ ]:
!python train_end_padding.py 4_operands_addition_plain.txt

Streaming output truncated to the last 5000 lines.
 53% 44/83 [00:01<00:01, 32.77it/s]Skipping y_hat=499
Skipping y_hat=1301
Skipping y_hat=999
Skipping y_hat=999
Skipping y_hat=1098
Skipping y_hat=1000
Skipping y_hat=1369
 58% 48/83 [00:01<00:01, 32.23it/s]Skipping y_hat=797
Skipping y_hat=1099
Skipping y_hat=666
Skipping y_hat=498
Skipping y_hat=1001
Skipping y_hat=999
Skipping y_hat=940
Skipping y_hat=1080
Skipping y_hat=1499
 63% 52/83 [00:01<00:00, 31.77it/s]Skipping y_hat=1006
 72% 60/83 [00:01<00:00, 32.70it/s]Skipping y_hat=700
Skipping y_hat=1099
 77% 64/83 [00:01<00:00, 32.75it/s]Skipping y_hat=298
Skipping y_hat=490
Skipping y_hat=699
 82% 68/83 [00:02<00:00, 32.55it/s]Skipping y_hat=1903
Skipping y_hat=1010
Skipping y_hat=1460
Skipping y_hat=938
Skipping y_hat=1084
Skipping y_hat=1797
 92% 76/83 [00:02<00:00, 33.93it/s]Skipping y_hat=2304
Skipping y_hat=2198
Skipping y_hat=2792
Skipping y_hat=1497
Skipping y_hat=1690
Skipping y_hat=1462
Skipping y_hat=1119
Skipping y_hat=90

## Reproduce 2 Operands 0-99 Multiplication (Their Code, Their Data)

In [ ]:
%cd /content/drive/MyDrive/teaching_arithmetic
%pwd   # verify you’re in the right place
!ls    # should show train.py, 4_operands_addition.txt, etc.

/content/drive/MyDrive/teaching_arithmetic
bench.py		  prompts
config			  __pycache__
config2			  README.md
config_gpt2		  result_analysis.ipynb
configurator.py		  run
data			  run_gpt2
evaluate_additions.py	  sample_addition_fewshot2.py
evaluate_models.py	  sample_addition_fewshot.py
LICENSE			  sample_addition.py
main_utils.py		  sample.py
matrix_completion	  test_addition.py
meta_all_ascii_chars.pkl  train_mixed.py
model.py		  train_multi_task.py
myStart.ipynb		  train_no527.py
out			  train.py
out2			  utils
plots			  wandb


In [ ]:
!python train.py config2/multiplication/plain/train_addition_bal.py

Streaming output truncated to the last 5000 lines.
correct: 66*13=858
outputs(x):  94*32=3948
4*
wrong  : 94*32=3948
correct: 94*32=3008
outputs(x):  14*66=1008
84
wrong  : 14*66=1008
correct: 14*66=924
outputs(x):  23*70=1980
59
wrong  : 23*70=1980
correct: 23*70=1610
outputs(x):  32*19=570
52*
wrong  : 32*19=570
correct: 32*19=608
outputs(x):  72*13=806
15*
wrong  : 72*13=806
correct: 72*13=936
outputs(x):  39*10=350
28*
wrong  : 39*10=350
correct: 39*10=390
outputs(x):  64*92=6256
77
wrong  : 64*92=6256
correct: 64*92=5888
outputs(x):  77*42=3654
73
wrong  : 77*42=3654
correct: 77*42=3234
outputs(x):  98*53=5144
14
wrong  : 98*53=5144
correct: 98*53=5194
outputs(x):  66*71=4752
33
wrong  : 66*71=4752
correct: 66*71=4686
outputs(x):  80*33=2673
66
wrong  : 80*33=2673
correct: 80*33=2640
outputs(x):  91*34=374
92*
wrong  : 91*34=374
correct: 91*34=3094
outputs(x):  47*40=1840
71
wrong  : 47*40=1840
correct: 47*40=1880
outputs(x):  27*50=1050
44
wrong  : 27*50=1050
correct: 27*50=1350


## 0-to-999 times 1-digit multiplication (balanced data)

Plain

In [ ]:
%cat config2/multiplication/plain/train_addition_bal.py

# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out2/multiplication_plain_0_to_999_times_1_digit_bal'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = True # override via command line if you like
wandb_project = 'multiplication'
wandb_run_name = 'plain_0_to_999_times_1_digit_bal'

data_type='text'
data_format='plain'
operator='*'
dataset = 'bal'
batch_size = 256
block_size = 256 # context of up to 256 previous characters
train_data_path = '0_to_999_times_1_digit_train_3000.txt'
# val_data_path = 'val.bin'
ckpt_path_name = 'ckpt.pt'
eval_addition = True
start = "FILE:data/bal/0_to_999_times_1_digit_test_3000.txt" #"FILE:data/bal/test_multiplication_7000.txt"
eval_addition_train = True
# start_train = "FILE:data/multiplication

In [ ]:
!python train.py config2/multiplication/plain/train_addition_bal.py

Streaming output truncated to the last 5000 lines.
wrong  : 847*7=6629
correct: 847*7=5929
outputs(x):  762*6=4532
52
wrong  : 762*6=4532
correct: 762*6=4572
outputs(x):  788*7=5526
23
wrong  : 788*7=5526
correct: 788*7=5516
outputs(x):  137*5=5
575*6
wrong  : 137*5=5
correct: 137*5=685
outputs(x):  162*7=1184
14
wrong  : 162*7=1184
correct: 162*7=1134
outputs(x):  823*3=249
752
wrong  : 823*3=249
correct: 823*3=2469
outputs(x):  104*9=90
458*
wrong  : 104*9=90
correct: 104*9=936
outputs(x):  179*4=756
303
wrong  : 179*4=756
correct: 179*4=716
outputs(x):  299*9=2601
54
wrong  : 299*9=2601
correct: 299*9=2691
outputs(x):  254*8=1232
76
wrong  : 254*8=1232
correct: 254*8=2032
outputs(x):  431*9=3987
70
wrong  : 431*9=3987
correct: 431*9=3879
outputs(x):  936*7=6524
23
wrong  : 936*7=6524
correct: 936*7=6552
outputs(x):  103*6=606
96*
wrong  : 103*6=606
correct: 103*6=618
outputs(x):  736*9=6804
21
wrong  : 736*9=6804
correct: 736*9=6624
outputs(x):  985*9=8855
60
wrong  : 985*9=8855
cor

Reverse

In [ ]:
!python train.py config2/multiplication/dollar_reverse/train_addition_dollar_reverse.py

Streaming output truncated to the last 5000 lines.
 33% 8/24 [00:00<00:00, 18.81it/s]outputs(x):  $239*9=1362$
$
wrong  : 239*9=2631
correct: 239*9=2151
outputs(x):  $457*6=2492$
$
wrong  : 457*6=2942
correct: 457*6=2742
outputs(x):  $533*8=4663$
$
wrong  : 533*8=3664
correct: 533*8=4264
outputs(x):  $119*6=415$
$4
wrong  : 119*6=514
correct: 119*6=714
outputs(x):  $525*8=0044$
$
wrong  : 525*8=4400
correct: 525*8=4200
outputs(x):  $103*4=218$
$1
wrong  : 103*4=812
correct: 103*4=412
outputs(x):  $396*8=8653$
$
wrong  : 396*8=3568
correct: 396*8=3168
outputs(x):  $529*8=2514$
$
wrong  : 529*8=4152
correct: 529*8=4232
outputs(x):  $788*7=6196$
$
wrong  : 788*7=6916
correct: 788*7=5516
outputs(x):  $220*8=0612$
$
wrong  : 220*8=2160
correct: 220*8=1760
outputs(x):  $383*9=7452$
$
wrong  : 383*9=2547
correct: 383*9=3447
outputs(x):  $627*6=2653$
$
wrong  : 627*6=3562
correct: 627*6=3762
outputs(x):  $670*9=0316$
$
wrong  : 670*9=6130
correct: 670*9=6030
outputs(x):  $386*7=2032$
$
wrong  

## 0_to_99999_times_1_digit

In [ ]:
%cd /content/drive/MyDrive/addition

/content/drive/MyDrive/addition


In [ ]:
%cat ./2_operands_mul_reversed.txt

# to edit: do an evaluation every {eval_interval} iterations
eval_interval = 100
eval_iters = 1
always_save_checkpoint = True


wandb_log = True # override via command line if you like
wandb_project = 'multiplication'

# to edit: wandb run name
wandb_run_name = '0_to_six_digit_times_1_digit_reverse'

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

# to edit: 'plain' or 'reverse' or 'algo_reasoning'
data_format='reverse'
operator='*'
dataset = 'bal'
batch_size = 512
block_size = 32 # context of up to 256 previous characters

# to edit: 'scratch' or 'resume', whether train from scratch or resume from a saved checkpoint
init_from = 'scratch'
# to edit: the checkpoint from which to resume training
ckpt_path_name = 'ckpt_2_operands_0_to_999_their_training_data_plain.pt'

# to edit: whether the result is reversed
reverse_c = True
eval_addition = True

analysis = False

# to edit: max number of digits in each operand
num_digit = 6

# to edit: maximum number of output to

In [ ]:
!python train_end_padding_more_early_eval.py 2_operands_mul_plain.txt

Streaming output truncated to the last 5000 lines.
 33% 9/27 [00:00<00:00, 21.49it/s]Skipping y_hat=4614088
Skipping y_hat=5860067
Skipping y_hat=736858
Skipping y_hat=4997944
Skipping y_hat=3806
Skipping y_hat=3119111
Skipping y_hat=1705928
Skipping y_hat=7221196
Skipping y_hat=3202621
Skipping y_hat=8519716
Skipping y_hat=7762022
Skipping y_hat=561017
Skipping y_hat=3180976
Skipping y_hat=5537022
Skipping y_hat=1649596
 44% 12/27 [00:00<00:00, 21.43it/s]Skipping y_hat=1972808
Skipping y_hat=7499377
Skipping y_hat=7798177
Skipping y_hat=1540029
Skipping y_hat=3392785
Skipping y_hat=813264
Skipping y_hat=2101484
Skipping y_hat=5395396
Skipping y_hat=5659273
Skipping y_hat=2620358
Skipping y_hat=793230
Skipping y_hat=2060166
Skipping y_hat=8989525
 56% 15/27 [00:00<00:00, 21.63it/s]Skipping y_hat=6252119
Skipping y_hat=5070236
Skipping y_hat=3271094
Skipping y_hat=2469998
Skipping y_hat=3597003
Skipping y_hat=875801
Skipping y_hat=8787691
Skipping y_hat=2403988
Skipping y_hat=5033699
Sk

In [ ]:
!python train_end_padding_more_early_eval.py 2_operands_mul_reversed.txt

Streaming output truncated to the last 5000 lines.
 56% 15/27 [00:00<00:00, 22.39it/s]Skipping y_hat=7321969
Skipping y_hat=6854776
Skipping y_hat=6033799
Skipping y_hat=716578
Skipping y_hat=772714
Skipping y_hat=5819248
 67% 18/27 [00:00<00:00, 22.14it/s]Skipping y_hat=5896313
Skipping y_hat=5579144
Skipping y_hat=6778727
Skipping y_hat=1884482
 78% 21/27 [00:00<00:00, 22.11it/s]Skipping y_hat=788759
 89% 24/27 [00:01<00:00, 22.92it/s]Skipping y_hat=865711
100% 27/27 [00:01<00:00, 22.75it/s]
accuracy of 3000 examples: 2956/3000 (98.53333333333333%)

Test Results:
0_to_six_digit_times_1_digit_bal_test: 98.53%

iter 635: train loss 0.9523, val loss 0.9454
Using precomputed batches
Max number of tokens 8.
  0% 0/27 [00:00<?, ?it/s]Skipping y_hat=3341304
Skipping y_hat=1380320
Skipping y_hat=2101535
Skipping y_hat=6728158
 11% 3/27 [00:00<00:01, 22.07it/s]Skipping y_hat=7639084
Skipping y_hat=6148430
Skipping y_hat=2556265
Skipping y_hat=5903245
Skipping y_hat=4393040
Skipping y_hat=5877

## 2 Operands 0 to 999 Uniform

In [ ]:
%cd /content/drive/MyDrive/addition

/content/drive/MyDrive/addition


In [ ]:
%cat 2_operands_addition_plain.txt

# to edit: do an evaluation every {eval_interval} iterations
eval_interval = 100
eval_iters = 1
always_save_checkpoint = True


wandb_log = True # override via command line if you like
wandb_project = 'addition'

# to edit: wandb run name
wandb_run_name = '2_operands_0_to_999_uniform_gold_by_reading_plain'

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

# to edit: 'plain' or 'reverse' or 'algo_reasoning'
data_format='plain'
operator='+'
dataset = 'bal'
batch_size = 512
block_size = 32 # context of up to 256 previous characters

# to edit: 'scratch' or 'resume', whether train from scratch or resume from a saved checkpoint
init_from = 'scratch'
iter_num = 0
# to edit: the checkpoint from which to resume training
ckpt_path_name = 'ckpt_2_operands_0_to_999_their_training_data_plain.pt'

# to edit: whether the result is reversed
reverse_c = False
eval_addition = True

analysis = False

# to edit: max number of digits in each operand
num_digit = 3

# to edit: maximum 

In [ ]:
%cat ./evaluation.py

from main_utilities import *
from tqdm.auto import tqdm
import torch
import numpy as np
import random
import math
import os
import pandas as pd
import csv


def get_abc_new(abc: str, zero_pad=False, reverse_c=False, binary=False, mode: str = "compute_gold"):
    """Unified parser: mode='compute_gold' computes the groudtruth on the fly;
       mode='read_gold_as_str' reads the groundtruth from the evaluation files (testing, validation) to do string matching.
    Returns either
      (operands_str, result_int, operation)            # v1
    or
      (operands_str, result_int, result_str, operation)  # v2
    """
    if '+' in abc:
        operation = '+'
    elif '-' in abc:
        operation = '-' 
    elif '*' in abc:
        operation = '*'
    else:
        print(f'operation not found, abc: {abc}')
        return None, None, None

    # Split the input string into parts
    parts = abc.split('=')
    if len(parts) != 2:
        print(f'Invalid format, expected "a+b+c...=result", got:

In [ ]:
%cat ./train.py

import os
import pickle
import requests
import numpy as np
import random
from tqdm import tqdm
import copy
import time

from contextlib import nullcontext

import numpy as np
import pandas as pd
import torch
import yaml
from torch.utils.data import Dataset, DataLoader
import wandb
import torch.nn.functional as F
import math

from model import GPTConfig, GPT
from main_utilities import *
from evaluation import *
from statistical_measurements import *

import re

def create_meta_for_addition(data):
    """Create metadata for addition data."""
    # Define the vocabulary for addition problems
    # This includes digits, operators, equals sign, and newline
    chars = sorted(list(set(data)))
    vocab_size = len(chars)
    # Create encoder and decoder dictionaries
    stoi = {ch: i for i, ch in enumerate(chars)}
    itos = {i: ch for i, ch in enumerate(chars)}
    
    meta = {
        'vocab_size': vocab_size,
        'vocab': chars,
        'stoi': stoi,
        'itos': itos
    }
    ret

In [ ]:
!python train.py 2_operands_addition_plain.txt

Streaming output truncated to the last 5000 lines.
Skipping y_hat=888
Skipping y_hat=63333
Skipping y_hat=63333
Skipping y_hat=63333
Skipping y_hat=63333
Skipping y_hat=63333
Skipping y_hat=63333
Skipping y_hat=63333
Skipping y_hat=88056
Skipping y_hat=88056
Skipping y_hat=88056
Skipping y_hat=88056
Skipping y_hat=88056
Skipping y_hat=88056
Skipping y_hat=88056
Skipping y_hat=
Skipping y_hat=
Skipping y_hat=
Skipping y_hat=
Skipping y_hat=
Skipping y_hat=
Skipping y_hat=
Skipping y_hat=
Skipping y_hat=
Skipping y_hat=
Skipping y_hat=
Skipping y_hat=
Skipping y_hat=
Skipping y_hat=
Skipping y_hat=03633
Skipping y_hat=03633
Skipping y_hat=03633
Skipping y_hat=03633
Skipping y_hat=03633
Skipping y_hat=03633
Skipping y_hat=03633
Skipping y_hat=
Skipping y_hat=
Skipping y_hat=
Skipping y_hat=
Skipping y_hat=
Skipping y_hat=
Skipping y_hat=
Skipping y_hat=997
Skipping y_hat=997
Skipping y_hat=997
Skipping y_hat=997
Skipping y_hat=997
Skipping y_hat=997
Skipping y_hat=997
Skipping y_hat=
Skip

In [ ]:
!python train.py 2_operands_addition_reversed.txt

Streaming output truncated to the last 5000 lines.
Skipping y_hat=616
Skipping y_hat=420
Skipping y_hat=689
Skipping y_hat=822
Skipping y_hat=182
Skipping y_hat=196
Skipping y_hat=732
Skipping y_hat=750
Skipping y_hat=229
Skipping y_hat=189
Skipping y_hat=178
Skipping y_hat=748
Skipping y_hat=86
Skipping y_hat=320
Skipping y_hat=340
Skipping y_hat=760
Skipping y_hat=133
Skipping y_hat=200
Skipping y_hat=862
Skipping y_hat=3
Skipping y_hat=411
Skipping y_hat=639
Skipping y_hat=742
Skipping y_hat=174
Skipping y_hat=804
Skipping y_hat=89
Skipping y_hat=722
Skipping y_hat=170
Skipping y_hat=676
Skipping y_hat=41
Skipping y_hat=145
Skipping y_hat=706
Skipping y_hat=132
Skipping y_hat=138
Skipping y_hat=248
Skipping y_hat=40
Skipping y_hat=722
Skipping y_hat=91
Skipping y_hat=774
Skipping y_hat=77
Skipping y_hat=185
Skipping y_hat=285
Skipping y_hat=882
Skipping y_hat=73
Skipping y_hat=78
Skipping y_hat=91
Skipping y_hat=74
100% 25/25 [00:00<00:00, 30.26it/s]
accuracy of 3000 examples: 2916/

## 2 Operands Output w/o Leading Digit

In [ ]:
%cd /content/drive/MyDrive/addition
%ls

/content/drive/MyDrive/addition
0_to_99999_times_1_digit/
0_to_six_digit_times_1_digit/
2_operands_0_to_999_balanced_digit_plain/
2_operands_0_to_999_balanced_digit_reversed/
2_operands_0_to_999_output_wo_leading_digit/
2_operands_0_to_999_their_data/
2_operands_0_to_999_their_training_data_only_more_eval/
2_operands_0_to_999_their_training_our_testing/
2_operands_0_to_999_uniform/
2_operands_0_to_999_uniform_using_new_code/
2_operands_3_digit_output_1000+/
2_operands_3_digit_output_padding/
2_operands_addition_plain.txt
2_operands_addition_reversed.txt
2_operands_mul_plain.txt
2_operands_mul_reversed.txt
3_operands_addition.txt
4_operands_0_to_999_balanced_digit/
4_operands_0_to_999_uniform_wo_padding/
4_operands_0_to_999_uniform_w_padding/
4_operands_3_digit_uniform_output_padding/
4_operands_addition_3_digit.txt
4_operands_addition_plain.txt
4_operands_addition_reversed.txt
5_operands_addition.txt
6_operands_0_to_999_balanced_digit/
6_operands_addition_reversed.txt
7_operands_additi

In [ ]:
%cat 2_operands_addition_plain.txt

# to edit: do an evaluation every {eval_interval} iterations
eval_interval = 100
eval_iters = 1
always_save_checkpoint = True


wandb_log = True # override via command line if you like
wandb_project = 'addition'

# to edit: wandb run name
wandb_run_name = '2_operands_0_to_999_output_wo_leading_digit_plain'

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

# to edit: 'plain' or 'reverse' or 'algo_reasoning'
data_format='plain'
operator='+'
dataset = 'bal'
batch_size = 512
block_size = 32 # context of up to 256 previous characters

# to edit: 'scratch' or 'resume', whether train from scratch or resume from a saved checkpoint
init_from = 'scratch'
# to edit: the checkpoint from which to resume training
ckpt_path_name = 'ckpt_2_operands_0_to_999_their_training_data_plain.pt'

# to edit: whether the result is reversed
reverse_c = False
eval_addition = True

analysis = False

# to edit: max number of digits in each operand
num_digit = 3

# to edit: maximum number of out

In [ ]:
%cat ./evaluation_by_reading_groundtruth.py

from main_utilities import *
from tqdm.auto import tqdm
import torch
import numpy as np
import random
import math
import os
import pandas as pd
import csv


def get_abc_new(abc: str, zero_pad=False, reverse_ab=False, binary=False):
    """Parse addition expression and return operands and result.
    Args:
        abc: String in format "a+b+c...=result"
        zero_pad: Whether to remove zero padding
        reverse_ab: Whether to reverse operands
        binary: Whether operands are in binary
    Returns:
        tuple: (operands_str, result_int, result_str, operation)
    """
    if '+' in abc:
        operation = '+'
    elif '-' in abc:
        operation = '-' 
    elif '*' in abc:
        operation = '*'
    else:
        print(f'operation not found, abc: {abc}')
        return None, None, None

    # Split the input string into parts
    parts = abc.split('=')
    if len(parts) != 2:
        print(f'Invalid format, expected "a+b+c...=result", got: {abc}')
        return None, Non

In [ ]:
!python train_with_eval_by_reading.py 2_operands_addition_plain.txt

Streaming output truncated to the last 5000 lines.
Skipping y_hat=476
Skipping y_hat=476
Skipping y_hat=476
Skipping y_hat=476
Skipping y_hat=476
Skipping y_hat=476
Skipping y_hat=476
Skipping y_hat=175
Skipping y_hat=175
Skipping y_hat=175
Skipping y_hat=175
Skipping y_hat=175
Skipping y_hat=175
Skipping y_hat=175
Skipping y_hat=654
Skipping y_hat=654
Skipping y_hat=654
Skipping y_hat=654
Skipping y_hat=654
Skipping y_hat=654
Skipping y_hat=654
Skipping y_hat=189
Skipping y_hat=189
Skipping y_hat=189
Skipping y_hat=189
Skipping y_hat=189
Skipping y_hat=189
Skipping y_hat=189
Skipping y_hat=819
Skipping y_hat=819
Skipping y_hat=819
Skipping y_hat=819
Skipping y_hat=819
Skipping y_hat=819
Skipping y_hat=819
Skipping y_hat=107
Skipping y_hat=107
Skipping y_hat=107
Skipping y_hat=107
Skipping y_hat=107
Skipping y_hat=107
Skipping y_hat=107
Skipping y_hat=489
Skipping y_hat=489
Skipping y_hat=489
Skipping y_hat=489
Skipping y_hat=489
Skipping y_hat=489
Skipping y_hat=489
Skipping y_hat=130

In [ ]:
!python train_with_eval_by_reading.py 2_operands_addition_reversed.txt

Streaming output truncated to the last 5000 lines.
Skipping y_hat=674
Skipping y_hat=674
Skipping y_hat=674
Skipping y_hat=674
Skipping y_hat=674
Skipping y_hat=674
Skipping y_hat=674
Skipping y_hat=571
Skipping y_hat=571
Skipping y_hat=571
Skipping y_hat=571
Skipping y_hat=571
Skipping y_hat=571
Skipping y_hat=571
Skipping y_hat=456
Skipping y_hat=456
Skipping y_hat=456
Skipping y_hat=456
Skipping y_hat=456
Skipping y_hat=456
Skipping y_hat=456
Skipping y_hat=981
Skipping y_hat=981
Skipping y_hat=981
Skipping y_hat=981
Skipping y_hat=981
Skipping y_hat=981
Skipping y_hat=981
Skipping y_hat=918
Skipping y_hat=918
Skipping y_hat=918
Skipping y_hat=918
Skipping y_hat=918
Skipping y_hat=918
Skipping y_hat=918
Skipping y_hat=701
Skipping y_hat=701
Skipping y_hat=701
Skipping y_hat=701
Skipping y_hat=701
Skipping y_hat=701
Skipping y_hat=701
Skipping y_hat=984
Skipping y_hat=984
Skipping y_hat=984
Skipping y_hat=984
Skipping y_hat=984
Skipping y_hat=984
Skipping y_hat=984
Skipping y_hat=031

## 4 Operands 0-to-999 Uniform with Padding

In [ ]:
%cat 4_operands_addition_reversed.txt

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
cat: 4_operands_addition_reversed.txt: Transport endpoint is not connected


In [ ]:
!python train_with_eval_by_reading.py 4_operands_addition_plain.txt

Streaming output truncated to the last 5000 lines.
Skipping y_hat=1810
Skipping y_hat=1810
Skipping y_hat=1810
Skipping y_hat=1810
Skipping y_hat=1810
Skipping y_hat=1810
Skipping y_hat=2253
Skipping y_hat=2253
Skipping y_hat=2253
Skipping y_hat=2253
Skipping y_hat=2253
Skipping y_hat=2253
Skipping y_hat=2253
Skipping y_hat=2135
Skipping y_hat=2135
Skipping y_hat=2135
Skipping y_hat=2135
Skipping y_hat=2135
Skipping y_hat=2135
Skipping y_hat=2135
Skipping y_hat=1772
Skipping y_hat=1772
Skipping y_hat=1772
Skipping y_hat=1772
Skipping y_hat=1772
Skipping y_hat=1772
Skipping y_hat=1772
Skipping y_hat=2491
Skipping y_hat=2491
Skipping y_hat=2491
Skipping y_hat=2491
Skipping y_hat=2491
Skipping y_hat=2491
Skipping y_hat=2491
Skipping y_hat=2923
Skipping y_hat=2923
Skipping y_hat=2923
Skipping y_hat=2923
Skipping y_hat=2923
Skipping y_hat=2923
Skipping y_hat=2923
Skipping y_hat=2597
Skipping y_hat=2597
Skipping y_hat=2597
Skipping y_hat=2597
Skipping y_hat=2597
Skipping y_hat=2597
Skipping 

In [ ]:
!python train_with_eval_by_reading.py 4_operands_addition_reversed.txt

Streaming output truncated to the last 5000 lines.
Skipping y_hat=5252
Skipping y_hat=5252
Skipping y_hat=5252
Skipping y_hat=5252
Skipping y_hat=5252
Skipping y_hat=3742
Skipping y_hat=3742
Skipping y_hat=3742
Skipping y_hat=3742
Skipping y_hat=3742
Skipping y_hat=3742
Skipping y_hat=3742
Skipping y_hat=6881
Skipping y_hat=6881
Skipping y_hat=6881
Skipping y_hat=6881
Skipping y_hat=6881
Skipping y_hat=6881
Skipping y_hat=6881
Skipping y_hat=5231
Skipping y_hat=5231
Skipping y_hat=5231
Skipping y_hat=5231
Skipping y_hat=5231
Skipping y_hat=5231
Skipping y_hat=5231
Skipping y_hat=5981
Skipping y_hat=5981
Skipping y_hat=5981
Skipping y_hat=5981
Skipping y_hat=5981
Skipping y_hat=5981
Skipping y_hat=5981
Skipping y_hat=1932
Skipping y_hat=1932
Skipping y_hat=1932
Skipping y_hat=1932
Skipping y_hat=1932
Skipping y_hat=1932
Skipping y_hat=1932
Skipping y_hat=6041
Skipping y_hat=6041
Skipping y_hat=6041
Skipping y_hat=6041
Skipping y_hat=6041
Skipping y_hat=6041
Skipping y_hat=6041
Skipping 

## With More Statistical Measurement

In [ ]:
%cd /content/drive/MyDrive/addition

/content/drive/MyDrive/addition


In [ ]:
%cat train_additional_measurement.py

import os
import pickle
import requests
import numpy as np
import random
from tqdm import tqdm
import copy
import time

from contextlib import nullcontext

import numpy as np
import pandas as pd
import torch
import yaml
from torch.utils.data import Dataset, DataLoader
import wandb
import torch.nn.functional as F
import math

from model import GPTConfig, GPT
from main_utilities import *
from evaluation_by_reading_groundtruth import *
from statistical_measurements import *

import re

def create_meta_for_addition(data):
    """Create metadata for addition data."""
    # Define the vocabulary for addition problems
    # This includes digits, operators, equals sign, and newline
    chars = sorted(list(set(data)))
    vocab_size = len(chars)
    # Create encoder and decoder dictionaries
    stoi = {ch: i for i, ch in enumerate(chars)}
    itos = {i: ch for i, ch in enumerate(chars)}
    
    meta = {
        'vocab_size': vocab_size,
        'vocab': chars,
        'stoi': stoi,
        'it

In [ ]:
%cat 4_operands_addition_reversed.txt

# to edit: do an evaluation every {eval_interval} iterations
eval_interval = 2000
eval_iters = 1
always_save_checkpoint = True


wandb_log = True # override via command line if you like
wandb_project = 'addition'

# to edit: wandb run name
wandb_run_name = '4_operands_0_to_999_output_wo_leading_digit_mi_reverse'

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

# to edit: 'plain' or 'reverse' or 'algo_reasoning'
data_format='reverse'
operator='+'
dataset = 'bal'
batch_size = 512
block_size = 32 # context of up to 256 previous characters

# to edit: 'scratch' or 'resume', whether train from scratch or resume from a saved checkpoint
init_from = 'scratch'
iter_num = 0
# to edit: the checkpoint from which to resume training
ckpt_path_name = 'ckpt_iter_19000_acc.pt'

# to edit: whether the result is reversed
reverse_c = True
eval_addition = True

analysis = False

# to edit: num of digits in each operand
num_digit = 3

# to edit: maximum number of output tokens (includ

In [ ]:
%cat statistical_measurements.py

import os
import pickle
import numpy as np
import random
from tqdm import tqdm
import copy

import numpy as np
import pandas as pd
import torch
import yaml
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import math
import sys

def encode_addition(text, meta):
    """Encode text to tensor using the metadata."""
    return torch.tensor([meta['stoi'][c] for c in text], dtype=torch.long)

def decode_addition(tensor, meta):
    """Decode tensor to text using the metadata."""
    if isinstance(tensor, torch.Tensor):
        return ''.join([meta['itos'][i.item()] for i in tensor])
    else:
        return ''.join([meta['itos'][i] for i in tensor])

def token_to_numeric(tensor, meta):
    """Convert tensor to numeric digits."""
    # Build lookup tensor
    lookup_tensor = torch.empty(len(meta["vocab"]), dtype=torch.long)
    for i, s in enumerate(meta["vocab"]):
        if s.isdigit():
            lookup_tensor[i] = int(s)
    return lookup_tensor[tensor]  # 

In [ ]:
!python train_additional_measurement.py 2_operands_addition_plain.txt

In [ ]:
!python train_additional_measurement.py 2_operands_addition_reversed.txt

Streaming output truncated to the last 5000 lines.
Skipping y_hat=3601
Skipping y_hat=3601
Skipping y_hat=3601
Skipping y_hat=6740
Skipping y_hat=6740
Skipping y_hat=6740
Skipping y_hat=6740
Skipping y_hat=6740
Skipping y_hat=6740
Skipping y_hat=6740
Skipping y_hat=5711
Skipping y_hat=5711
Skipping y_hat=5711
Skipping y_hat=5711
Skipping y_hat=5711
Skipping y_hat=5711
Skipping y_hat=5711
Skipping y_hat=4561
Skipping y_hat=4561
Skipping y_hat=4561
Skipping y_hat=4561
Skipping y_hat=4561
Skipping y_hat=4561
Skipping y_hat=4561
Skipping y_hat=9810
Skipping y_hat=9810
Skipping y_hat=9810
Skipping y_hat=9810
Skipping y_hat=9810
Skipping y_hat=9810
Skipping y_hat=9810
Skipping y_hat=9180
Skipping y_hat=9180
Skipping y_hat=9180
Skipping y_hat=9180
Skipping y_hat=9180
Skipping y_hat=9180
Skipping y_hat=9180
Skipping y_hat=7011
Skipping y_hat=7011
Skipping y_hat=7011
Skipping y_hat=7011
Skipping y_hat=7011
Skipping y_hat=7011
Skipping y_hat=7011
Skipping y_hat=9841
Skipping y_hat=9841
Skipping 

In [ ]:
!python train_additional_measurement.py 4_operands_addition_reversed.txt

Streaming output truncated to the last 5000 lines.
Skipping y_hat=170
Skipping y_hat=170
Skipping y_hat=170
Skipping y_hat=170
Skipping y_hat=170
Skipping y_hat=170
Skipping y_hat=170
Skipping y_hat=301
Skipping y_hat=301
Skipping y_hat=301
Skipping y_hat=301
Skipping y_hat=301
Skipping y_hat=301
Skipping y_hat=301
Skipping y_hat=454
Skipping y_hat=454
Skipping y_hat=454
Skipping y_hat=454
Skipping y_hat=454
Skipping y_hat=454
Skipping y_hat=454
Skipping y_hat=119
Skipping y_hat=119
Skipping y_hat=119
Skipping y_hat=119
Skipping y_hat=119
Skipping y_hat=119
Skipping y_hat=119
Skipping y_hat=104
Skipping y_hat=104
Skipping y_hat=104
Skipping y_hat=104
Skipping y_hat=104
Skipping y_hat=104
Skipping y_hat=104
Skipping y_hat=700
Skipping y_hat=700
Skipping y_hat=700
Skipping y_hat=700
Skipping y_hat=700
Skipping y_hat=700
Skipping y_hat=700
Skipping y_hat=686
Skipping y_hat=686
Skipping y_hat=686
Skipping y_hat=686
Skipping y_hat=686
Skipping y_hat=686
Skipping y_hat=686
Skipping y_hat=779

# Subtraction

In [ ]:
%cat train.py

import os
import pickle
import requests
import numpy as np
import random
from tqdm import tqdm
import copy
import time

from contextlib import nullcontext

import numpy as np
import pandas as pd
import torch
import yaml
from torch.utils.data import Dataset, DataLoader
import wandb
import torch.nn.functional as F
import math

from model import GPTConfig, GPT
from main_utilities import *
from evaluation import *
from statistical_measurements import *

import re

def create_meta_for_addition(data):
    """Create metadata for addition data."""
    # Define the vocabulary for addition problems
    # This includes digits, operators, equals sign, and newline
    chars = sorted(list(set(data)))
    vocab_size = len(chars)
    # Create encoder and decoder dictionaries
    stoi = {ch: i for i, ch in enumerate(chars)}
    itos = {i: ch for i, ch in enumerate(chars)}
    
    meta = {
        'vocab_size': vocab_size,
        'vocab': chars,
        'stoi': stoi,
        'itos': itos
    }
    ret

In [ ]:
!python train.py 2_operands_subtraction_reversed.txt

Streaming output truncated to the last 5000 lines.
Skipping y_hat=-206
Skipping y_hat=-206
Skipping y_hat=-206
Skipping y_hat=-206
Skipping y_hat=-206
Skipping y_hat=-206
Skipping y_hat=-206
Skipping y_hat=+488
Skipping y_hat=+488
Skipping y_hat=+488
Skipping y_hat=+488
Skipping y_hat=+488
Skipping y_hat=+488
Skipping y_hat=+488
Skipping y_hat=+415
Skipping y_hat=+415
Skipping y_hat=+415
Skipping y_hat=+415
Skipping y_hat=+415
Skipping y_hat=+415
Skipping y_hat=+415
Skipping y_hat=-418
Skipping y_hat=-418
Skipping y_hat=-418
Skipping y_hat=-418
Skipping y_hat=-418
Skipping y_hat=-418
Skipping y_hat=-418
Skipping y_hat=-598
Skipping y_hat=-598
Skipping y_hat=-598
Skipping y_hat=-598
Skipping y_hat=-598
Skipping y_hat=-598
Skipping y_hat=-598
Skipping y_hat=+1
Skipping y_hat=+1
Skipping y_hat=+1
Skipping y_hat=+1
Skipping y_hat=+1
Skipping y_hat=+1
Skipping y_hat=+1
Skipping y_hat=-603
Skipping y_hat=-603
Skipping y_hat=-603
Skipping y_hat=-603
Skipping y_hat=-603
Skipping y_hat=-603
Ski

## 4 Operand Addition First Phase Change

In [ ]:
%cat configuration_files/4_operands_addition_reversed.txt

# to edit: do an evaluation every {eval_interval} iterations
eval_interval = 1000
eval_iters = 1
always_save_checkpoint = True


wandb_log = True # override via command line if you like
wandb_project = 'addition'

# to edit: wandb run name
wandb_run_name = '4_operands_0_to_999_first_phase_change'

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

# to edit: 'plain' or 'reverse' or 'algo_reasoning'
data_format='reverse'
operator='+'
dataset = 'bal'
batch_size = 512
block_size = 32 # context of up to 256 previous characters

# to edit: 'scratch' or 'resume', whether train from scratch or resume from a saved checkpoint
init_from = 'scratch'
iter_num = 0
# to edit: the checkpoint from which to resume training
ckpt_path_name = 'ckpt_iter_19000_acc.pt'

# to edit: whether the result is reversed
reverse_c = True
eval_addition = True

analysis = False

# to edit: num of digits in each operand
num_digit = 3

# to edit: maximum number of output tokens (including '$')
max_new

In [ ]:
%cd ../

/content/drive/MyDrive/addition


In [ ]:
!python train.py 4_operands_addition_reversed.txt

Streaming output truncated to the last 5000 lines.
Skipping y_hat=2058
Skipping y_hat=2058
Skipping y_hat=2058
Skipping y_hat=2058
Skipping y_hat=2058
Skipping y_hat=2058
Skipping y_hat=2058
Skipping y_hat=1603
Skipping y_hat=1603
Skipping y_hat=1603
Skipping y_hat=1603
Skipping y_hat=1603
Skipping y_hat=1603
Skipping y_hat=1603
Skipping y_hat=2364
Skipping y_hat=2364
Skipping y_hat=2364
Skipping y_hat=2364
Skipping y_hat=2364
Skipping y_hat=2364
Skipping y_hat=2364
Skipping y_hat=1198
Skipping y_hat=1198
Skipping y_hat=1198
Skipping y_hat=1198
Skipping y_hat=1198
Skipping y_hat=1198
Skipping y_hat=1198
Skipping y_hat=1187
Skipping y_hat=1187
Skipping y_hat=1187
Skipping y_hat=1187
Skipping y_hat=1187
Skipping y_hat=1187
Skipping y_hat=1187
Skipping y_hat=1114
Skipping y_hat=1114
Skipping y_hat=1114
Skipping y_hat=1114
Skipping y_hat=1114
Skipping y_hat=1114
Skipping y_hat=1114
Skipping y_hat=1052
Skipping y_hat=1052
Skipping y_hat=1052
Skipping y_hat=1052
Skipping y_hat=1052
Skipping 

## 4 Operand Addition Scratchpad

In [ ]:
%cat configuration_files/4_operands_addition_scratchpad.txt

# to edit: do an evaluation every {eval_interval} iterations
eval_interval = 1000
eval_iters = 1
always_save_checkpoint = True


wandb_log = True # override via command line if you like
wandb_project = 'addition'

# to edit: wandb run name
wandb_run_name = '4_operands_0_to_999_scratchpad'

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

# to edit: 'plain' or 'reverse' or 'scratchpad'
data_format='scratchpad'
operator='+'
dataset = 'bal'
batch_size = 512
block_size = 64 # context of up to 256 previous characters

# to edit: 'scratch' or 'resume', whether train from scratch or resume from a saved checkpoint
init_from = 'scratch'
iter_num = 0
# to edit: the checkpoint from which to resume training
ckpt_path_name = 'ckpt_iter_19000_acc.pt'

# to edit: num of digits in each operand
num_digit = 3

# to edit: maximum number of output tokens (including '$')
max_new_tokens = 50

drop_leading_digit = False

# whether need zero‐padding
zero_pad = False

# ===== Learning Rat

In [ ]:
%cat evaluation.py

from main_utilities import *
from tqdm.auto import tqdm
import torch
import numpy as np
import random
import math
import os
import pandas as pd
import csv


def get_abc_new(abc: str, zero_pad=False, data_format="plain", binary=False, mode: str = "compute_gold"):
    """Unified parser: mode='compute_gold' computes the groudtruth on the fly;
       mode='read_gold_as_str' reads the groundtruth from the evaluation files (testing, validation) to do string matching.
    Returns either
      (operands_str, result_int, operation)            # v1
    or
      (operands_str, result_int, result_str, operation)  # v2
    """
    if '+' in abc:
        operation = '+'
    elif '-' in abc:
        operation = '-' 
    elif '*' in abc:
        operation = '*'
    else:
        print(f'operation not found, abc: {abc}')
        return None, None, None

    # Split the input string into parts
    parts = abc.split('=')
    if len(parts) != 2:
        print(f'Invalid format, expected "a+b+c...=result", 

In [ ]:
!python train.py 4_operands_addition_scratchpad.txt

Overriding config with ./configuration_files/4_operands_addition_scratchpad.txt:
# to edit: do an evaluation every {eval_interval} iterations
eval_interval = 1000
eval_iters = 1
always_save_checkpoint = True


wandb_log = True # override via command line if you like
wandb_project = 'addition'

# to edit: wandb run name
wandb_run_name = '4_operands_0_to_999_scratchpad'

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

# to edit: 'plain' or 'reverse' or 'scratchpad'
data_format='scratchpad'
operator='+'
dataset = 'bal'
batch_size = 512
block_size = 64 # context of up to 256 previous characters

# to edit: 'scratch' or 'resume', whether train from scratch or resume from a saved checkpoint
init_from = 'scratch'
iter_num = 0
# to edit: the checkpoint from which to resume training
ckpt_path_name = 'ckpt_iter_19000_acc.pt'

# to edit: num of digits in each operand
num_digit = 3

# to edit: maximum number of output tokens (including '$')
max_new_tokens = 50

drop_leading_

## 4 Operand Max

In [5]:
%cat configuration_files/4_operands_max.txt

# to edit: do an evaluation every {eval_interval} iterations
eval_interval = 50
eval_iters = 1
always_save_checkpoint = True


wandb_log = True # override via command line if you like
wandb_project = 'addition'

# to edit: wandb run name
wandb_run_name = '4_operands_max_balanced_digit'

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

# to edit: 'plain' or 'reverse' or 'scratchpad' or 'max'
data_format='max'
operator=''
dataset = 'bal'
batch_size = 512
block_size = 32 # context of up to 256 previous characters

# to edit: 'scratch' or 'resume', whether train from scratch or resume from a saved checkpoint
init_from = 'scratch'
iter_num = 0
# to edit: the checkpoint from which to resume training
ckpt_path_name = 'ckpt_iter_19000_acc.pt'

# to edit: num of digits in each operand
num_digit = 3

# to edit: maximum number of output tokens (including '$')
max_new_tokens = 10

drop_leading_digit = False

# whether need zero‐padding
zero_pad = False

# ===== Learning Rate 

In [4]:
%cat train.py

import os
import pickle
import requests
import numpy as np
import random
from tqdm import tqdm
import copy
import time

from contextlib import nullcontext

import numpy as np
import pandas as pd
import torch
import yaml
from torch.utils.data import Dataset, DataLoader
import wandb
import torch.nn.functional as F
import math

from model import GPTConfig, GPT
from main_utilities import *
from evaluation import *
from statistical_measurements import *

import re

def create_meta_for_addition(data):
    """Create metadata for addition data."""
    # Define the vocabulary for addition problems
    # This includes digits, operators, equals sign, and newline
    chars = sorted(list(set(data)))

    # ensure special pad/eos token exists
    if '$' not in chars:
        chars.append('$')
        chars = sorted(chars)
        
    vocab_size = len(chars)
    # Create encoder and decoder dictionaries
    stoi = {ch: i for i, ch in enumerate(chars)}
    itos = {i: ch for i, ch in enumerate(chars)}

In [5]:
%cat model.py

import torch
import torch.nn as nn
from torch.nn import functional as F

import math
import inspect
from dataclasses import dataclass

def new_gelu(x):
    """
    Implementation of the GELU activation function currently in Google BERT repo (identical to OpenAI GPT).
    Reference: Gaussian Error Linear Units (GELU) paper: https://arxiv.org/abs/1606.08415
    """
    return 0.5 * x * (1.0 + torch.tanh(math.sqrt(2.0 / math.pi) * (x + 0.044715 * torch.pow(x, 3.0))))

class LayerNorm(nn.Module):
    """ LayerNorm but with an optional bias. PyTorch doesn't support simply bias=False """

    def __init__(self, ndim, bias):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

    def forward(self, input):
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)
    
class CasualSelfAttention(nn.Module):
  def __init__(self,config):
    super().__init__()
    assert con

In [6]:
!python train.py 4_operands_max.txt

Streaming output truncated to the last 5000 lines.
Skipping y_hat=7608
Skipping y_hat=7608
Skipping y_hat=7608
Skipping y_hat=7608
Skipping y_hat=7608
Skipping y_hat=9836
Skipping y_hat=9836
Skipping y_hat=9836
Skipping y_hat=9836
Skipping y_hat=9836
Skipping y_hat=9836
Skipping y_hat=9836
Skipping y_hat=6726
Skipping y_hat=6726
Skipping y_hat=6726
Skipping y_hat=6726
Skipping y_hat=6726
Skipping y_hat=6726
Skipping y_hat=6726
Skipping y_hat=8243
Skipping y_hat=8243
Skipping y_hat=8243
Skipping y_hat=8243
Skipping y_hat=8243
Skipping y_hat=8243
Skipping y_hat=8243
Skipping y_hat=9038
Skipping y_hat=9038
Skipping y_hat=9038
Skipping y_hat=9038
Skipping y_hat=9038
Skipping y_hat=9038
Skipping y_hat=9038
Skipping y_hat=7521
Skipping y_hat=7521
Skipping y_hat=7521
Skipping y_hat=7521
Skipping y_hat=7521
Skipping y_hat=7521
Skipping y_hat=7521
Skipping y_hat=5097
Skipping y_hat=5097
Skipping y_hat=5097
Skipping y_hat=5097
Skipping y_hat=5097
Skipping y_hat=5097
Skipping y_hat=5097
Skipping 

## 4 Operand Sorting

In [2]:
%cat configuration_files/4_operands_sorting.txt

# to edit: do an evaluation every {eval_interval} iterations
eval_interval = 1000
eval_iters = 1
always_save_checkpoint = True


wandb_log = True # override via command line if you like
wandb_project = 'addition'

# to edit: wandb run name
wandb_run_name = '4_operands_sorting_balanced_digit_symbol'

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

# to edit: 'plain' or 'reverse' or 'scratchpad' or 'max' or 'sorting'
data_format='sorting'
operator=''
dataset = 'bal'
batch_size = 512
block_size = 64 # context of up to 256 previous characters

# to edit: 'scratch' or 'resume', whether train from scratch or resume from a saved checkpoint
init_from = 'scratch'
iter_num = 0
# to edit: the checkpoint from which to resume training
ckpt_path_name = 'ckpt_iter_19000_acc.pt'

# to edit: num of digits in each operand
num_digit = 4

# to edit: maximum number of output tokens (including '$')
max_new_tokens = 16

drop_leading_digit = False

# whether need zero‐padding
zero_pad =

In [ ]:
!python train.py 4_operands_sorting.txt

Streaming output truncated to the last 5000 lines.
Skipping y_hat=W,X,V
Skipping y_hat=X,V,W
Skipping y_hat=X,V,W
Skipping y_hat=X,V,W
Skipping y_hat=X,V,W
Skipping y_hat=X,V,W
Skipping y_hat=X,V,W
Skipping y_hat=X,V,W
Skipping y_hat=F,E,G
Skipping y_hat=F,E,G
Skipping y_hat=F,E,G
Skipping y_hat=F,E,G
Skipping y_hat=F,E,G
Skipping y_hat=F,E,G
Skipping y_hat=F,E,G
Skipping y_hat=Y,X,W
Skipping y_hat=Y,X,W
Skipping y_hat=Y,X,W
Skipping y_hat=Y,X,W
Skipping y_hat=Y,X,W
Skipping y_hat=Y,X,W
Skipping y_hat=Y,X,W
Skipping y_hat=K,L,J
Skipping y_hat=K,L,J
Skipping y_hat=K,L,J
Skipping y_hat=K,L,J
Skipping y_hat=K,L,J
Skipping y_hat=K,L,J
Skipping y_hat=K,L,J
Skipping y_hat=N,P,O
Skipping y_hat=N,P,O
Skipping y_hat=N,P,O
Skipping y_hat=N,P,O
Skipping y_hat=N,P,O
Skipping y_hat=N,P,O
Skipping y_hat=N,P,O
Skipping y_hat=D,C,B
Skipping y_hat=D,C,B
Skipping y_hat=D,C,B
Skipping y_hat=D,C,B
Skipping y_hat=D,C,B
Skipping y_hat=D,C,B
Skipping y_hat=D,C,B
Skipping y_hat=X,V,W
Skipping y_hat=X,V,W
Skip